### Instalação dos pacotes em Julia

In [ ]:
import Pkg; Pkg.add("JuMP")
import Pkg; Pkg.add("HiGHS")
import Pkg; Pkg.add("GLPK")
import Pkg; Pkg.add("Clp")
import Pkg; Pkg.add("DataFrames")

### Importando às bibliotecas que serão usadas para resolver os problemas de PLI

In [1]:
using JuMP
import HiGHS
import GLPK
using Clp
using Ipopt
using DataFrames

### Funções Auxiliares

Vamos criar uma função que resolva todos os casos na forma:

$$ \begin{align} \min / \max \; & c^T x \\
s.t \; & Ax = b \\
& Cx \leq d \\
& x \geq 0 
\end{align}$$

E outras funções para facilitar a visualização dos resultados.

In [2]:
# Resolve um problema de programação linear inteira nas formas acima
function LPP_Cases(opt, index, weights, A, b, C, d)
    model = Model(GLPK.Optimizer)
    p = length(weights)
    # Se x tem dimensão (i,j)
    if typeof(index) == Tuple{Int64, Int64}
        i, j = index
        @variable(model, x[1:i, 1:j] >= 0)
        x = reshape(x', p, 1)
    # Se x é um vetor
    else
        @variable(model, x[1:index] >= 0)
    end
    # Se houver restrições de igualdade
    if A != 0
        for m in 1:length(b)
            @constraint(model, (A*x)[m] == b[m])
        end
    end
    # Se houver restrições de desigualdade (<=)
    if C != 0
        for n in 1:length(d)
            @constraint(model, (C*x)[n] <= d[n])
        end
    end
    # Se a função objetivo for de maximização
    if opt == "Max"
        @objective(model, Max, sum(weights[k] * x[k] for k in 1:p))
        objetivo = "(Lucro Máximo)"
    # Se a função objetivo for de minimização
    else
        @objective(model, Min, sum(weights[k] * x[k] for k in 1:p))
        objetivo = "(Custo Mínimo)"
    end
    optimize!(model)
    println("Função Objetivo: $opt ", objective_function(model))
    println("Valor Objetivo $objetivo: ", objective_value(model))
    println("Valores Encontradas: ")
    for i in all_variables(model)
        println(i, " = ", value(i))
    end
end

# Printa o valor das variáveis de decisão que foram encontradas
function print_var(x)
    # Se for uma Matrix
    if typeof(x) == Matrix{VariableRef}
        for i in 1:size(x,1)
            for j in 1:size(x,2)
                if value(x[i,j]) - round(value(x[i,j])) < 0.5
                    println(x[i,j], " = ", round(Int, value(x[i,j])))
                else
                    println(x[i,j], " = ", value(x[i,j]))
                end
            end
        end
    # Se for um vetor
    elseif typeof(x) == Vector{VariableRef}
        for i in 1:length(x)
            println(x[i]," = ", value(x[i]))
        end
    # Se for apenas uma variável
    elseif typeof(x) == VariableRef
        println(x, " = ", value(x))
    else 
        println("Tipo de variável não suportado.")
    end
end

function print_model(model::Model, vars)
    # Se não tem muitas restrições, printa o modelo
    if sum(num_constraints(model, F, S) for (F, S) in list_of_constraint_types(model)) < 15
        println("Resolvendo o LPP:", "\n", model)
    end
    println("Valor Objetivo: ", objective_value(model))
    println("Valores Encontrados: ")
    for var in vars
        print_var(var)
    end
end

# Constrói um DataFrame com os nomes das variáveis e seus valores
function build_df(names::Vector{String}, values::Vector{Vector})
    n_cols, n_rows = length(names), length(values[1])
    if n_cols != length(values)
        println("Número de colunas e linhas não correspondem.")
        return
    end
    results = Matrix{Any}(undef, n_rows, n_cols)
    # Preenche a matriz com os valores das variáveis
    for i in 1:n_cols
        for j in 1:n_rows
            if (typeof(values[i][j]) == VariableRef) || (typeof(values[i][j]) == AffExpr)
                results[j,i] = round(Int, value(values[i][j]))
            else
                results[j,i] = values[i][j]
            end
        end
    end
    df = DataFrame(results, names) 
    println(df)
    println()
end

build_df (generic function with 1 method)

### Lista 1

**Questão 1** Uma fábrica possui duas usinas $U_1$ e $U_2$. A usina $U_1$ dispõe de 400 unidades de um produto e a $U_2$ de 300 unidades do mesmo produto. A fábrica tem três clientes $E_1 , E_2$ e $E_3$ cujas *demandas* respectivas para o produto são 100 unidades para $E_1 , 200$ unidades para $E_2$ e 300 unidades para $E_3$. Os *custos de transportes* são resumidos na tabela seguinte:

\begin{array}{|c|c|c|c|}
\hline
 & \text{E1} & \text{E2} & \text{E3} \\
\hline
\text{U1} & 1 & 1.5 & 3.5 \\
\hline
\text{U2} & 2 & 1 & 2 \\
\hline
\end{array}

Por exemplo, cada unidade fornecida a $E_1$ a partir de $U_2$ custa 2 reais. Como obter um sistema de distribuição ótimo.

In [3]:
model = Model(GLPK.Optimizer)
set_silent(model)

i = 2 # Número de Usinas
j = 3 # Número de Clientes

weights = [1, 1.5, 3.5, 2, 1, 2]

@variable(model, x[1:i, 1:j] >= 0)
# As demandas respectivas para o produto são:
@constraint(model, x[1,1] + x[2,1]   == 100) #E_1
@constraint(model, x[1,2] + x[2,2]  == 200) # E_2
@constraint(model, x[1,3] + x[2,3]  == 300) # E_3
# Restições de Capacidade
@constraint(model, x[1,1] + x[1,2] + x[1,3] <= 400) # U_1
@constraint(model, x[2,1] + x[2,2] + x[2,3] <= 300) # U_2
# Função Objetivo
@objective(model, Min, sum(weights[n] * x[div(n-1,3)+1, mod(n-1,3)+1] for n in 1:length(weights)))

optimize!(model)
print_model(model, x)

Resolvendo o LPP:
Min x[1,1] + 1.5 x[1,2] + 3.5 x[1,3] + 2 x[2,1] + x[2,2] + 2 x[2,3]
Subject to
 x[1,1] + x[2,1] == 100
 x[1,2] + x[2,2] == 200
 x[1,3] + x[2,3] == 300
 x[1,1] + x[1,2] + x[1,3] <= 400
 x[2,1] + x[2,2] + x[2,3] <= 300
 x[1,1] >= 0
 x[2,1] >= 0
 x[1,2] >= 0
 x[2,2] >= 0
 x[1,3] >= 0
 x[2,3] >= 0

Valor Objetivo: 1000.0
Valores Encontrados: 
x[1,1] = 100.0
x[2,1] = 0.0
x[1,2] = 200.0
x[2,2] = 0.0
x[1,3] = 0.0
x[2,3] = 300.0


Com a função...

In [4]:
# Igualdades Ax = b
A = [1 0 0 1 0 0;
0 1 0 0 1 0;
0 0 1 0 0 1]
b = [100;200;300]
# Desigualdades Cx <= d
C = [1 1 1 0 0 0;
 0 0 0 1 1 1]
d = [400; 300]
# Pesos e índices	
weights = [1 1.5 3.5 2 1 2]
i = 2
j = 3

LPP_Cases("Min", (i,j), weights, A, b, C, d)

Função Objetivo: Min x[1,1] + 1.5 x[1,2] + 3.5 x[1,3] + 2 x[2,1] + x[2,2] + 2 x[2,3]
Valor Objetivo (Custo Mínimo): 1000.0
Valores Encontradas: 
x[1,1] = 100.0
x[2,1] = 0.0
x[1,2] = 200.0
x[2,2] = 0.0
x[1,3] = 0.0
x[2,3] = 300.0


**Questão 2.**
Numa usina, produzimos 2 tipos de produtos a partir de 3 fertilizantes. O **produto I**, composto por *1 Kg* de nitratos e *2 Kg* de sal de potássio é vendido por R\$ 7, e o **produto II**, composto de *1 Kg* de nitratos, *1 Kg* de fosfatos e *3 Kg* de sal de potássio é  por R\$ 9. *Sobram* no estoque *8 Kg* de nitratos, *4 Kg* de fosfatos e *19 Kg* de sal de potássio.

1. Qual quantidade de cada produto a empresa tem que produzir para *maximizar o lucro*? 

In [5]:
model = Model(HiGHS.Optimizer)
set_silent(model)
# Quantidades produzidas pela empresa do produto I e II
@variable(model, p[1:i] >= 0)
# Função Objetivo
@objective(model, Max, 7*p[1] + 9*p[2])
# Restrição de Recurso
@constraint(model, p[1] + p[2] <= 8)
@constraint(model, 2*p[1] + 3*p[2] <= 4)
@constraint(model, p[2] <= 19)
# Otimiza o modelo
optimize!(model)
print_model(model, p)

Resolvendo o LPP:
Max 7 p[1] + 9 p[2]
Subject to
 p[1] + p[2] <= 8
 2 p[1] + 3 p[2] <= 4
 p[2] <= 19
 p[1] >= 0
 p[2] >= 0

Valor Objetivo: 14.0
Valores Encontrados: 
p[1] = 2.0
p[2] = 0.0


Com a função...

In [6]:
# Igualdades Ax = b
A, b = 0, 0
# Desigualdades Cx <= d
C = [1 1 ;
    2 3 ; 
    0 2]
d = [8; 4; 19]
# Pesos e índices	
weights = [7 9]
i = 2

LPP_Cases("Max", i, weights, A, b, C, d)

Função Objetivo: Max 7 x[1] + 9 x[2]
Valor Objetivo (Lucro Máximo): 14.0
Valores Encontradas: 
x[1] = 2.0
x[2] = 0.0


2. Uma cooperativa agricola quer negociar (i.e., *minimizar*) o preço de *1 Kg* de cada componente para comprar a granel todos os fertilizantes do estoque. Como determinar os preços para que a venda a granel seja pelo-menos tão lucrativa como a venda dos produtos?

In [7]:
model = Model(HiGHS.Optimizer)
set_silent(model)
# Preços de 1 KG de NO3, KCL, PO4
@variable(model, x[1:j] >= 0)
# Função Objetivo
@objective(model, Min, 9*x[1] + 19*x[2] + 4*x[3])
# Restrição de Lucro
@constraint(model, 9*x[1] + 2*x[2] >= 7)
@constraint(model, x[1] + 3*x[2] + x[3] >= 9)
# Otimiza o modelo
optimize!(model)
print_model(model, x)

Resolvendo o LPP:
Min 9 x[1] + 19 x[2] + 4 x[3]
Subject to
 9 x[1] + 2 x[2] >= 7
 x[1] + 3 x[2] + x[3] >= 9
 x[1] >= 0
 x[2] >= 0
 x[3] >= 0

Valor Objetivo: 39.888888888888886
Valores Encontrados: 
x[1] = 0.7777777777777778
x[2] = 0.0
x[3] = 8.222222222222221


Com a função...

In [8]:
# Igualdades Ax = b
A, b = 0, 0
# Desigualdades Cx >= d é o mesmo que -Cx <= -d
C = [9 2 0 ;
    1 3 1]
d = [7; 9]
# Pesos e índices	
weights = [9 19 4]
i = 3

LPP_Cases("Min", i, weights, A, b, -C, -d)

Função Objetivo: Min 9 x[1] + 19 x[2] + 4 x[3]
Valor Objetivo (Custo Mínimo): 39.888888888888886
Valores Encontradas: 
x[1] = 0.7777777777777778
x[2] = 0.0
x[3] = 8.222222222222221


**Questão 3** Um modelo de carro é montado em 3 usinas situadas nas cidades \(V1\), \(V2\) e \(V3\). O motor destes modelos é fornecido por duas outras usinas situadas nas cidades \(U1\) e \(U2\). As usinas de montagem precisam de pelo menos 5, 4 e 3 motores, respectivamente. Cada usina pode fornecer no máximo 6 motores. A direção da empresa quer minimizar o custo de transporte dos motores entre os dois sítios de fábrica e os três sítios de montagem. Os custos unitários (por motor transportado) para todos os itinerários possíveis são:


\begin{array}{|c|c|c|c|}
\hline
 & \text{V1} & \text{V2} & \text{V3} \\
\hline
\text{U1} & 38 & 27 & 48 \\
\hline
\text{U2} & 37 & 58 & 45 \\
\hline
\end{array}

Como minimizar o custo total de transporte respeitando a oferta e a demanda?

In [9]:
modelo = Model(GLPK.Optimizer)
# Define os valores de oferta e demanda
oferta, demanda = [6, 6], [5, 4, 3]
# Define os custos de transporte
custos = [38 27 48;
          37 58 45]

# Define as variáveis de decisão
@variable(modelo, x[1:2, 1:3] >= 0)
# Define as restrições de oferta
for i in 1:2
    @constraint(modelo, sum(x[i, j] for j in 1:3) <= oferta[i])
end
# Define as restrições de demanda
for j in 1:3
    @constraint(modelo, sum(x[i, j] for i in 1:2) >= demanda[j])
end

# Define a função objetivo para minimizar o custo total de transporte
@objective(modelo, Min, sum(custos[i, j] * x[i, j] for i in 1:2, j in 1:3))
# Resolve o problema de otimização
optimize!(modelo)
# Verifica se o problema é resolvido de forma ótima
@assert termination_status(modelo) == MOI.OPTIMAL
print_model(modelo, x) # Custo Total de Transporte

Resolvendo o LPP:
Min 38 x[1,1] + 27 x[1,2] + 48 x[1,3] + 37 x[2,1] + 58 x[2,2] + 45 x[2,3]
Subject to
 x[1,1] + x[2,1] >= 5
 x[1,2] + x[2,2] >= 4
 x[1,3] + x[2,3] >= 3
 x[1,1] + x[1,2] + x[1,3] <= 6
 x[2,1] + x[2,2] + x[2,3] <= 6
 x[1,1] >= 0
 x[2,1] >= 0
 x[1,2] >= 0
 x[2,2] >= 0
 x[1,3] >= 0
 x[2,3] >= 0

Valor Objetivo: 430.0
Valores Encontrados: 
x[1,1] = 2.0
x[2,1] = 3.0
x[1,2] = 4.0
x[2,2] = 0.0
x[1,3] = 0.0
x[2,3] = 3.0


**Questão 4** Um sapateiro faz 6 sapatos por hora se fizer somente sapatos e 5 cintos por hora se fizer somente cintos. Ele *trabalha 10 horas por dia* e gasta **2 unidades de couro para fabricar 1 unidade de sapato** e **1 unidade de couro para fabricar 1 unidade de cinto**. Sabendo-se que o *total disponível de couro é de 78 unidades por dia* e que o #underline[lucro unitário por sapato é de 5 reais] e o de **cinto é de 4 reais**, pede-se: o modelo do sistema de produção diario do sapateiro, se o objetivo é maximizar seu lucro diario.

**Questão 5**
Certo fabricante de combustível para avião vende 2 tipos de combustível, A e B. O combustível de tipo A possui $25 %$ de gasolina $1 , 25 %$ de gasolina 2 e $50 %$ de gasolina 3 . 0 combustível B tem $50 %$ de gasolina 2 e $50 %$ de gasolina 3. Há disponível para produção 500 galões de gasolina 1 e 200 galões de cada gasolina 2 e 3. Os lucros pela venda dos combustíveis A e B são, respectivamente, 20 e 30 dólares. Quanto se deve fazer de cada combustível para se obter um lucro máximo? Formule e resolva o problema.

**Questão 6** Uma fábrica de petróleo deseja utilizar quatro tipos de petróleos para produzir três tipos de diesel: A, B, e C. \
A respeito dos tipos de petróleo, temos as seguintes informaões:

\begin{array}{|c|c|c|}
\hline
 \text{Tipo de Petróleo} & \text{Quant. max. disp. por dia} & \text{Custo (reais por baril)} \\
\hline
1 & 3000 & 3  \\
\hline
2 & 2000 & 6 \\
\hline
3 & 4000 & 4  \\
\hline 
4 & 1000 & 5  \\
\hline
\end{array}


O diesel A **não pode conter mais de** $30 %$ do petróleo do tipo 1 , #underline[nem mais de] $50 %$ do tipo 3 , mas **deve conter no mínimo** $40 %$ do tipo 2 . O preço de venda deste diesel é de 5.5 reais por baril.

O diesel B cujo preço de venda é 4.5 reais por baril, #underline[deverá ser composto de pelo menos] $10 %$ do tipo 2 mas **no máximo de** $50 %$ do tipo 1 .

O diesel C **não poderá conter mais de** $70 %$ do petróleo do tipo 1 e o seu preço de venda é de 3.5 reais por baril.

A fabrica gostaria de saber a quantidade de baris de cada tipo de petróleo que deveria ser utilizada na fabricação de cada um dos tipos de diesel para poder *maximizar seu lucro*.

In [12]:
model = Model(HiGHS.Optimizer)
set_silent(model)
i, j = 4, 3 # Número de tipos de petróleo e diesel 

# Quantidade do tipo i por dia pro Diesel j (1 -> A, 2 -> B, 3 -> C)
@variable(model, Q[1:i, 1:j] >= 0)

# Restrição da quantidade máxima disposta por dia de cada tipo de petróleo
@constraint(model, sum(Q[1,n] for n in 1:j) <= 3000)
@constraint(model, sum(Q[2,n] for n in 1:j) <= 2000)
@constraint(model, sum(Q[3,n] for n in 1:j) <= 4000)
@constraint(model, sum(Q[4,n] for n in 1:j) <= 1000)

# Restrições de quantidade de cada tipo de petróleo no Diesel A
@constraint(model, Q[1,1] <= 0.3 * sum(Q[m,1] for m in 1:i))
@constraint(model, Q[3,1] <= 0.5 * sum(Q[m,1] for m in 1:i))
@constraint(model, Q[2,1] >= 0.4 * sum(Q[m,1] for m in 1:i))
# Restrições de quantidade de cada tipo de petróleo no Diesel B
@constraint(model, Q[1,2] <= 0.5 * sum(Q[m,2] for m in 1:i))
@constraint(model, Q[2,2] >= 0.1 * sum(Q[m,2] for m in 1:i))
# Restrições de quantidade de cada tipo de petróleo no Diesel C
@constraint(model, Q[1,3] <= 0.7 * sum(Q[m,3] for m in 1:i))

# Função Objetivo
@objective(model, Max, 5.5 * sum(Q[m,1] for m in 1:i) + 4.5 * sum(Q[m,2] for m in 1:i) + 3.5 * sum(Q[m,3] for m in 1:i) - 3*(Q[1,1] + Q[1,2] + Q[1,3]) - 6*(Q[2,1] + Q[2,2] + Q[2,3]) - 4*(Q[3,1] + Q[3,2] + Q[3,3]) - 5*(Q[4,1] + Q[4,2] + Q[4,3]))

println("Função Objetiva: ", objective_function(model))
optimize!(model)
print_model(model, Q)

Função Objetiva: 2.5 Q[1,1] - 0.5 Q[2,1] + 1.5 Q[3,1] + 0.5 Q[4,1] + 1.5 Q[1,2] - 1.5 Q[2,2] + 0.5 Q[3,2] - 0.5 Q[4,2] + 0.5 Q[1,3] - 2.5 Q[2,3] - 0.5 Q[3,3] - 1.5 Q[4,3]
Valor Objetivo: 7166.666666666666
Valores Encontrados: 
Q[1,1] = 366.6666666666665
Q[2,1] = 1466.6666666666667
Q[3,1] = 1833.3333333333335
Q[4,1] = 0.0
Q[1,2] = 2633.3333333333335
Q[2,2] = 533.3333333333334
Q[3,2] = 2166.6666666666665
Q[4,2] = 0.0
Q[1,3] = 0.0
Q[2,3] = 0.0
Q[3,3] = 0.0
Q[4,3] = 0.0


### Lista 2

**1- Gestão da Produção de Eletricidade**

As usinas térmicas e hidroelétricas brasileiras podem ser agrupadas em 4 subsistemas intercambiando energia entre eles. Supomos que cada região contém uma usina hidroelétrica (agregação das usinas hidroelétricas deste subsistema) e um número arbitrário de usinas térmicas. Cada usina hidroelétrica funciona com um reservatório. 
Somente $80 %$ das afluências de uma região dada é armazenado nos reservatórios. O restante é diretamente convertido em eletricidade por usinas ao fil d’agua. O custo de produção da eletricidade com as usinas térmicas é uma função linear da produção enquanto ele é considerado nulo com usinas hidroelétricas. \
Cada usina tem uma capacidade de produção conhecida e os níveis dos reservatórios devem ficar entre determinados valores mínimos e máximos. Cada dia, a demanda dos clientes deve ser atendida, eventualmente comprando energia no mercado spot a um custo unitário mais alto que o maior custo unitario das térmicas.

Explique como determinar as produções diárias das usinas térmicas e hidroelétricas para o mês seguinte, de maneira a minimizar o custo e satisfazendo a demanda e as restrições de funcionamento das usinas.

**2 - Gestão de Carteiros**

Queremos investir $M$ reais em $n$ ativos financeiros. O retorno do ativo $i$ no período de investimento é $r_i$. Escrever um problema de otimização linear que permita determinar a quantidade de dinheiro a investir em cada ativo para maximizar o lucro. Qual é a solução deste problema?

**3 - Planificação da expansão da produção**
Consideramos o problema de expansão da capacidade de produção de uma usina produzindo $m$ produtos. Cada uma das $n$ máquinas é flexível e cada produto pode ser produzido por qualquer máquina. A máquina $j$ está agora disponivel para $h_j$ horas de funcionamento por semana e horas adicionais podem ser adquiridas num custo atualizado de $c_j$ por hora. O uso da máquina $j$ é limitado por uma cota superior de $u_j$ horas, por outra parte, uma revisão de $t_j$ horas da máquina $j$ é necessaria para cada hora de funcionamento. O tempo total gasto em revisão não pode ultrapassar $T$ horas. A taxa de produção do produto $i$ na maquina $j$ é $a_(i j)$, com um custo associado de $g_(i j)$ por hora.

Cada semana, a empresa deve satisfazer a demanda em cada um dos $m$ produtos. Cada unidade de produto $i$ não vendida acarreta um custo $p_i$. A empresa quer decidir quantas horas adicionais são necessárias para cada máquina com os dados a seguir:

In [55]:
n = 4 # Número de máquinas j
m = 3 # Número de produtos i
T = 100 # Número de horas disponíveis

p = vec([400,400,400])
c = vec([2.5, 3.75, 5.0, 3.0])
t = vec([0.08, 0.04, 0.03, 0.01])
h = vec([500,500,500,500])
u = vec([2000,2000,3000,3000])

a = [0.6 0.6 0.9 0.8;
         0.1 0.9 0.6 0.8;
         0.05 0.2 0.5 0.8]

g = [2.6 3.4 3.4 2.5;
         1.5 2.4 2.0 3.6;
         4.0 3.8 3.5 3.2]

d = vec([1800, 600, 3000]);

In [96]:
model = Model(HiGHS.Optimizer)
# H_i,j : Horas de funcionamento da máquina j dedicadas à produção do produto i
@variable(model, H[1:m, 1:n] >= 0)
# e_j : Quantidade de horas empregadas ao custo c_j
@variable(model, e[1:n] >= 0)

# Definimos x_i como a produção de um produto i: para cada máquina j, temos o produto da taxa de produção a_i,j e o tempo de funcionamento H_i,j
@expression(model, x[i=1:m], sum(a[i,j]*H[i,j] for j in 1:n))
# Cada máquina j tem um número inicial de horas disponíveis h_j e pode adquirir horas adicionais y_j, mas o total de horas utilizadas (horas iniciais + horas adicionais) não pode exceder u_j
for j in 1:n
    @constraint(model, h[j] + e[j] <= u[j])
end

# A soma do tempo de revisão de todas as máquinas não pode exceder T horas. Considerando que cada hora de funcionamento de uma máquina j requer t_j horas de revisão:
@constraint(model, sum(t[j]*(e[j] + h[j]) for j in 1:n) <= T)

# Definimos s_i como demanda não atendida.
@expression(model, s[i=1:m], d[i] - x[i])

# A produção de cada produto i por todas as máquinas deve atender ou exceder a demanda especificada
for i in 1:m
    @constraint(model, x[i] <=  d[i])
end

# A quantidade de horas extras empregadas ao custo c_j (e_j) somado às horas de funcionamento disponível por semana para a máquina j (h_j) deve ser igual ao total de horas de funcionamento dedicadas à produção do produto i (H_i,j)
for j in 1:n
    @constraint(model, e[j] + h[j] == sum(H[i,j] for i in 1:m))
end

# A função objetivo quer minimizar a função de custo total, que é a soma dos custos de produção e revisão
@objective(model, Min,
    sum(c[j]*e[j] for j in 1:n) +
    sum(g[i,j]*H[i,j] for i in 1:m, j in 1:n) +
    sum(p[i]*s[i] for i in 1:m)
);

In [97]:
set_silent(model)
optimize!(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))
println("Valores de Encontrados:")
print_var(H)
print_var(e)

Valor Objetivo (Custo Mínimo): 897281.25
Valores de Encontrados:
H[1,1] = 500
H[1,2] = 0
H[1,3] = 500
H[1,4] = 1312.5
H[2,1] = 0
H[2,2] = 500
H[2,3] = 0
H[2,4] = 0
H[3,1] = 0
H[3,2] = 0
H[3,3] = 0
H[3,4] = 1188
e[1] = 0.0
e[2] = 0.0
e[3] = 0.0
e[4] = 2000.0


É interessante visualizar estes resultados em um DataFrame.

In [106]:
build_df(["Produto i", "Demanda", "Demanda não atendida", "Produção M1", "Produção M2", "Produção M3", "Produção M4", "Produção Total"],
        [["Produto $i" for i in 1:m], d, s, [(H[i,1]*a[i,1]) for i in 1:m], [(H[i,2]*a[i,2]) for i in 1:m],
         [(H[i,3]*a[i,3]) for i in 1:m], [(H[i,4]*a[i,4]) for i in 1:m], x])

build_df(["Máquina j", "Custo (cj)", "H. Extras (ej)", "H. Revisão", "H.Totais", "H. Produto 1", "H. Produto 2", "H. Produto 3"],
        [["Máquina $j" for j in 1:n],c,e,
        [(e[j] + h[j])*t[j] for j in 1:n],
        [(e[j] + h[j]) for j in 1:n],
        H[1,:], H[2,:], H[3,:]])

3×8 DataFrame
 Row │ Produto i  Demanda  Demanda não atendida  Produção M1  Produção M2  Produção M3  Produção M4  Produção Total 
     │ Any        Any      Any                   Any          Any          Any          Any          Any            
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ Produto 1  1800     0                     300          0            450          1050         1800
   2 │ Produto 2  600      150                   0            450          0            0            450
   3 │ Produto 3  3000     2050                  0            0            0            950          950

4×8 DataFrame
 Row │ Máquina j  Custo (cj)  H. Extras (ej)  H. Revisão  H.Totais  H. Produto 1  H. Produto 2  H. Produto 3 
     │ Any        Any         Any             Any         Any       Any           Any           Any          
─────┼──────────────────────────────────────────────────────────────────────────────

### 4 - Gestão de contratos com opção de cancelamento

### 5 - Problema de "Unit commitment" 

### Problema da Mochila

In [19]:
n = 5; # Número de Itens
capacity = 10.0; # Capacidade da Mochila
profit = [5.0, 3.0, 2.0, 7.0, 4.0]; # Lucro de cada item
weight = [2.0, 8.0, 4.0, 2.0, 5.0]; # Peso de cada item

#model = Model(GLPK.Optimizer)
model = Model(HiGHS.Optimizer)

# Variáveis Binárias
@variable(model, x[1:n], Bin)
# Restrição de Capacidade
@constraint(model, sum(weight[i] * x[i] for i in 1:n) <= capacity)
# Função Objetivo
@objective(model, Max, sum(profit[i] * x[i] for i in 1:n))
print(model) # LPP

Max 5 x[1] + 3 x[2] + 2 x[3] + 7 x[4] + 4 x[5]
Subject to
 2 x[1] + 8 x[2] + 4 x[3] + 2 x[4] + 5 x[5] <= 10
 x[1] binary
 x[2] binary
 x[3] binary
 x[4] binary
 x[5] binary


In [20]:
set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
#solution_summary(model)
println("Valor Objetivo (Custo Mínimo): ", objective_value(model))
print("Itens Escolhidos: ", [i for i in 1:n if value(x[i]) == 1])

Valor Objetivo (Custo Mínimo): 16.0
Itens Escolhidos: [1, 4, 5]

### Prova A1

**Questão 1** Exercício 1. 2 pontos. Uma empresa pode escolher quatro tipos de líquidos: 8000 litros do líquido A ao custo unitario 5,50 \$, 4250 litros de B ao custo unitario $4,50 \$, 16000$ litros de C ao custo unitario $7,50 \$$, e 2000 litros de D ao custo unitario $11,25 \$$.

A empresa pode revender estes líquidos diretamente, sem transforma-los, e vendelos por $6 \$$ por litro.

Ela pode também elaborar as misturas $E, F$ e $G$. As misturas devem apresentar as características dadas na tabela 

\begin{array}{|c|c|c|c|c|}
\hline 
\text{Mistura} & \text{Líquido A} & \text{Líquido B} & \text{Líquido C} & \text{Líquido D} \\
\hline 
E & 30 \% & \text{Pelo menos } 10 \% & 40 \% & \text{No máximo } 5\% \\
\hline 
F & \text{Pelo menos } 25 \% & \text{No máximo } 20\% & 20 \% & \text{Pelo menos } 10 \% \\
\hline 
G & 20 \% & \text{Pelo menos } 15 \% & 40 \% & \text{No máximo } 20 \% \\
\hline
\end{array}

Table 1: Proporção de cada líquido num litro de mistura
As misturas se vendem respectivamente $11 \$, 15 \$$ e $14 \$$ por litro, e o mercado pode comprar todas as misturas produzidas.

A empresa é obrigada a produzir pelo menos 400 litros de E, pelo menos 800 litros de F e pelo menos 200 litros de G.

Enfim, misturando 2 partes de $\mathrm{G}$ com uma parte de E, podemos obter um produto $\mathrm{P}$ vendido $22 \$$ por litro e cuja demande é suficientemente grande para ser considerada ilimitada.

Modelar este problema por um problema de otimização linear dado que a empresa quer maximizar seu lucro.

In [21]:
model = Model(HiGHS.Optimizer)

i, j = 4, 3

# l_i: quantidade de líquido i revendido diretamente
@variable(model, l[1:i] >= 0)
# x_ij: quantidade de líquido i = 1,2,3,4 (A,B,C,D) usado na produção de j = 1,2,3 (E,F,G)
@variable(model, x[1:i, 1:j] >= 0)
# p: quantidade de líquido P produzido
@variable(model, p >= 0)

# make sum of x[m,j] <= 8000 as constraint
d = [8000, 4250, 16000, 2000]
for m in 1:i
    @constraint(model, l[m] + sum(x[m,j] for j in 1:j) <= d[m])
end

# Para produzir E, j=1
@constraint(model, x[1,1] == 0.3 * sum(x[m,1] for m in 1:i)) # A 
@constraint(model, x[2,1] >= 0.1 * sum(x[m,1] for m in 1:i)) # B
@constraint(model, x[3,1] == 0.4 * sum(x[m,1] for m in 1:i)) # C
@constraint(model, x[4,1] <= 0.05 * sum(x[m,1] for m in 1:i)) # D

# Para produzir F, j=2
@constraint(model, x[1,2] == 0.25 * sum(x[m,2] for m in 1:i)) # A
@constraint(model, x[2,2] <= 0.2 * sum(x[m,2] for m in 1:i)) # B
@constraint(model, x[3,2] == 0.2 * sum(x[m,2] for m in 1:i)) # C
@constraint(model, x[4,2] >= 0.1 * sum(x[m,2] for m in 1:i)) # D

# Para produzir G, j=3
@constraint(model, x[1,3] == 0.2 * sum(x[m,3] for m in 1:i)) # A
@constraint(model, x[2,3] >= 0.15 * sum(x[m,3] for m in 1:i)) # B
@constraint(model, x[3,3] == 0.4 * sum(x[m,3] for m in 1:i)) # C
@constraint(model, x[4,3] <= 0.2 * sum(x[m,3] for m in 1:i)) # D

# A produção de líquidos E,F,G deve suprir a obrigação da empresa
@constraint(model, sum(x[m,1] for m in 1:i) >= 400)
@constraint(model, sum(x[m,2] for m in 1:i) >= 800)
@constraint(model, sum(x[m,3] for m in 1:i) >= 200)

# O líquido P deve satisfazer as proporções especificadas de líquido E e G
@constraint(model, p/3 <= sum(x[m,1] for m in 1:i))
@constraint(model, 2*p/3 <= sum(x[m,3] for m in 1:i))

# Função Objetivo
@objective(model, Max, 6*(sum(l[m] for m in 1:i)) + 11*(sum(x[m,1] for m in 1:i) - p/3) + 15*(sum(x[m,2] for m in 1:i)) + 14*(sum(x[m,3] for m in 1:i) - 2*p/3)) + 22*p - 5.5*(l[1] + sum(x[1,n] for n in 1:j)) - 3.5*(l[2] + sum(x[2,j] for j in 1:j)) - 7.5*(l[3] + sum(x[3,j] for j in 1:j)) - 11.5*(l[4] + sum(x[4,j] for j in 1:j));

set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
print_model(model, [x, p, l])

Valor Objetivo: 304500.0
Valores Encontrados: 
x[1,1] = 120
x[1,2] = 200
x[1,3] = 2845
x[2,1] = 120
x[2,2] = 0
x[2,3] = 4130
x[3,1] = 160
x[3,2] = 160
x[3,3] = 5690
x[4,1] = 0
x[4,2] = 440
x[4,3] = 1560
p = 0.0
l[1] = 4835.0
l[2] = 0.0
l[3] = 9990.0
l[4] = 0.0


In [22]:
build_df(["Líquido i", "Revendido Diretamente", "Quant. Produção E", "Quant. Produção F", "Quant. Produção G"],
        [["Líquido $i" for i in ["A","B","C","D"]], l, x[:,1], x[:,2], x[:,3]])

print("Quantidade de Líquido P produzido: ", Int(value(p)))

4×5 DataFrame
 Row │ Líquido i  Revendido Diretamente  Quant. Produção E  Quant. Produção F  Quant. Produção G 
     │ Any        Any                    Any                Any                Any               
─────┼───────────────────────────────────────────────────────────────────────────────────────────
   1 │ Líquido A  4835                   120                200                2845
   2 │ Líquido B  0                      120                0                  4130
   3 │ Líquido C  9990                   160                160                5690
   4 │ Líquido D  0                      0                  440                1560

Quantidade de Líquido P produzido: 0

**Questão 2** Exercício 2. 1.75 pontos. A companhia energética Dark necessita realizar o planejamento energético para um novo prédio. A energia necessária é classificada em 3 categorias: (a) iluminação; (b) aquecimento ambiente; (c) aquecimento água. As necessidades mensais de cada categorias são:
\begin{array}{|c|c|}
\hline \text{Iluminação} & 20 \mathrm{MW} \\
\hline \text{Aquecimento ambiente} & 10 \mathrm{MW} \\
\hline \text{Aquecimento água} & 30 \mathrm{MW} \\
\hline
\end{array}
3 fontes de energia podem ser instaladas para suprir a energia necessária: (a) eletricidade; (b) painéis solares; (c) gás natural. O suprimento máximo mensal de energia de cada fonte é:
\begin{array}{|c|c|}
\hline 
\text{Eletricidade} & 50 \mathrm{MW} \\
\hline
\text{Painéis solares} & 50 \mathrm{MW} \\
\hline 
\text{Gás natural} & 20 \mathrm{MW} \\
\hline
\end{array}

A iluminação só pode ser suprida pela energia "eletricidade", a um custo de $R \$ 50$ por MW. As demais categorias ("aquecimento ambiente" e "aquecimento água") podem ser supridas por qualquer fonte. Os custos unitários de suprimento para estas categorias (em $\mathrm{R} \$ / \mathrm{MW}$ ) são os seguintes:
\begin{array}{|c|c|c|c|}
\hline & \text{Eletricidade} & \text{Gás natural} & \text{Painéis solares} \\
\hline \text{Aquecimento ambiente} & 90 & 60 & 30 \\
\hline \text{Aquecimento água} & 80 & 50 & 40 \\
\hline
\end{array}

O objetivo é minimizar o custo total de energia mensal do prédio. Escrever um programa linear para modelar este problema.

In [23]:
model = Model(HiGHS.Optimizer)

i, j = 3, 3
weights = [50, 90, 80, 0, 30, 40, 0, 60, 50]
# Quantidade de energia de i -> j, onde i=(1,2,3) são eletricidade, painéis solares e gás natural, e 
# j=(1,2,3) são iluminação, aquecimento ambiente, aquecimento água, respectivamente. 
@variable(model, x[1:i, 1:j] >= 0)
# As demandas respectivas para cada categoria devem ser atendidas
@constraint(model, x[1,1] == 20) # Iluminação
@constraint(model, x[1,2] + x[2,2] + x[3,2] == 10) # Aquecimento Ambiente
@constraint(model, x[1,3] + x[2,3] + x[3,3]  == 30) # Aquecimento Água
# Restrições de Capacidade
@constraint(model, x[1,1] + x[1,2] + x[1,3]  <= 50) # Eletricidade
@constraint(model, x[2,2] + x[2,3]  <= 50) # Painéis Solares
@constraint(model, x[3,2] + x[3,3]  <= 20) # Gás Natural

# Função Objetivo
@objective(model, Min, sum(weights[n] * x[div(n-1,3)+1, mod(n-1,3)+1] for n in 1:9))

50 x[1,1] + 90 x[1,2] + 80 x[1,3] + 30 x[2,2] + 40 x[2,3] + 60 x[3,2] + 50 x[3,3]

In [24]:
set_silent(model)
optimize!(model)
@assert is_solved_and_feasible(model)
print_model(model, x)

Valor Objetivo: 2500.0
Valores Encontrados: 
x[1,1] = 20.0
x[2,1] = 0.0
x[3,1] = 0.0
x[1,2] = 0.0
x[2,2] = 10.0
x[3,2] = 0.0
x[1,3] = 0.0
x[2,3] = 30.0
x[3,3] = 0.0


Com a função...

In [25]:
A = [1 0 0 0 0 0 0 0 0;
0 1 0 0 1 0 0 1 0;
0 0 1 0 0 1 0 0 1]
b = [20; 10; 30]

C = [1 1 1 0 0 0 0 0 0 ;
0 0 0 0 1 1 0 0 0 ;
0 0 0 0 0 0 0 1 1]
d = [50; 50; 20];

LPP_Cases("Min", (i,j), weights, A, b, C, d)

Função Objetivo: Min 50 x[1,1] + 90 x[1,2] + 80 x[1,3] + 30 x[2,2] + 40 x[2,3] + 60 x[3,2] + 50 x[3,3]
Valor Objetivo (Custo Mínimo): 2500.0
Valores Encontradas: 
x[1,1] = 20.0
x[2,1] = 0.0
x[3,1] = 0.0
x[1,2] = 0.0
x[2,2] = 10.0
x[3,2] = 0.0
x[1,3] = 0.0
x[2,3] = 30.0
x[3,3] = 0.0


Note que iluminação recebe apenas de eletricidade, e aquecimento ambiente/água recebem apenas de painéis solares, o que era esperado, já que são a opção mais barata, logo para minimizar os custos basta comprar a demanda necessária de eletricidade e instalar painéis solares suficientes para suprir a demanda de aquecimento.

In [26]:
build_df(["Quantidade de Energia de", "Eletricidade", "Painéis Solares", "Gás Natural"],
        [["Para Iluminação", "Para Aquecimento Ambiente", "Para Aquecimento Água"], x[1,:], x[2,:], x[3,:]])

3×4 DataFrame
 Row │ Quantidade de Energia de   Eletricidade  Painéis Solares  Gás Natural 
     │ Any                        Any           Any              Any         
─────┼───────────────────────────────────────────────────────────────────────
   1 │ Para Iluminação            20            0                0
   2 │ Para Aquecimento Ambiente  0             10               0
   3 │ Para Aquecimento Água      0             30               0

